In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def get_data(file_path):
    # get the data from the csv file
    df = pd.read_csv(file_path, sep='\t', usecols=['path', 'accents', 'up_votes', 'down_votes'])

    # drop rows with nan values
    non_nan_df = df.dropna()

    return non_nan_df

In [ ]:
def filter_data(df, upvote_threshold=8, num_accents=10, additional_accents=None):
    if additional_accents is None:
        additional_accents = []

    # drop rows with accents other than the ones in top num_accents and additional accents
    top_accents = df['accents'].value_counts().head(num_accents).index
    top_df = df[df['accents'].isin(top_accents) | df['accents'].isin(additional_accents)]

    # drop rows with diffrence between upvotes and downvotes less than threshold
    top_df = top_df[(top_df['up_votes'] - top_df['down_votes']) >= upvote_threshold]

    # drop up_votes and down_votes columns and add new column 'accent' with values as integers
    top_df.drop(columns=['up_votes', 'down_votes'], inplace=True)
    top_df['accent'] = top_df['accents'].astype('category').cat.codes

    return top_df

In [ ]:
def split_data(dataframe, train_size=0.9, val_size=0.3):
    train_df = pd.DataFrame()
    val_df = pd.DataFrame()
    test_df = pd.DataFrame()

    for accent in dataframe['accent'].unique():
        accent_df = dataframe[dataframe['accent'] == accent]

        accent_train_df = accent_df.sample(frac=train_size, random_state=42)
        accent_val_test_df = accent_df.drop(accent_train_df.index)
        accent_val_df = accent_val_test_df.sample(frac=val_size, random_state=42)
        accent_test_df = accent_val_test_df.drop(accent_val_df.index)

        train_df = pd.concat([train_df, accent_train_df])
        val_df = pd.concat([val_df, accent_val_df])
        test_df = pd.concat([test_df, accent_test_df])

    return train_df, val_df, test_df

In [ ]:
def save_class_labels(dataframe, data_dir):
    # save value counts of accents
    accent_counts = dataframe['accents'].value_counts()
    accent_counts.to_csv(os.path.join(data_dir, 'accent_counts.csv'))

    # save class labels
    class_labels = accent_counts.index
    class_labels_df = pd.Series(class_labels)
    class_labels_df.to_csv(os.path.join(data_dir, 'class_labels.csv'), index=False)

In [ ]:
def plot_data(dataframe, title='Accent Distribution', save_path=None):
    accent_counts = dataframe['accents'].value_counts()
    plt.figure(figsize=(8, 6))
    accent_counts.plot(kind='barh')
    plt.ylabel('Accent')
    plt.xlabel('Count')
    plt.title(title)
    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")

    plt.show()

In [ ]:
def main():
    # Constants
    file_path = r"../../data/metadata/validated.tsv"
    results_dir = r"../../results/"
    upvote_threshold = 3
    num_accents = 10
    additional_accents = []

    # get the data
    df = get_data(file_path)

    # filter the data
    filtered_df = filter_data(df, upvote_threshold, num_accents, additional_accents)

    # split the data
    train_df, val_df, test_df = split_data(filtered_df)

    print(f"Train size: {train_df.shape[0]} samples")
    print(f"Val size: {val_df.shape[0]} samples")
    print(f"Test size: {test_df.shape[0]} samples")

    # plot the data distribution and save plots pngs
    plot_data(filtered_df, 'Accent Distribution for Filtered',
              os.path.join(results_dir, 'accent_distribution_filtered.png'))
    plot_data(train_df, 'Accent Distribution for Train', os.path.join(results_dir, 'accent_distribution_train.png'))
    plot_data(val_df, 'Accent Distribution for Val', os.path.join(results_dir, 'accent_distribution_val.png'))
    plot_data(test_df, 'Accent Distribution for Test', os.path.join(results_dir, 'accent_distribution_test.png'))

    # save class labels
    save_class_labels(filtered_df, results_dir)

In [ ]:
main()